In [1]:
import pandas as pd


In [2]:
transactions = pd.read_csv('/content/cleaned_data.csv')
clusters = pd.read_csv('/content/customer_clusters.csv')

transactions['InvoiceDate'] = pd.to_datetime(transactions['InvoiceDate'])
transactions.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
data = transactions.merge(
    clusters[['CustomerID', 'Cluster']],
    on='CustomerID',
    how='inner'
)


In [4]:
cluster_top_products = (
    data
    .groupby(['Cluster', 'StockCode', 'Description'])['Quantity']
    .sum()
    .reset_index()
    .sort_values(['Cluster', 'Quantity'], ascending=[True, False])
)


In [5]:
TOP_N = 5

top_products_per_cluster = (
    cluster_top_products
    .groupby('Cluster')
    .head(TOP_N)
)


In [6]:
customer_purchases = (
    data
    .groupby('CustomerID')['StockCode']
    .apply(set)
    .to_dict()
)


In [7]:
recommendations = []

for _, row in clusters.iterrows():
    customer_id = row['CustomerID']
    cluster = row['Cluster']

    purchased = customer_purchases.get(customer_id, set())

    cluster_products = top_products_per_cluster[
        top_products_per_cluster['Cluster'] == cluster
    ]

    recommended = cluster_products[
        ~cluster_products['StockCode'].isin(purchased)
    ]

    for _, rec in recommended.iterrows():
        recommendations.append({
            'CustomerID': customer_id,
            'Cluster': cluster,
            'Recommended_StockCode': rec['StockCode'],
            'Recommended_Product': rec['Description']
        })


In [8]:
recommendation_df = pd.DataFrame(recommendations)
recommendation_df.head()


,CustomerID,Cluster,Recommended_StockCode,Recommended_Product
0,12346.0,1.0,23843,"PAPER CRAFT , LITTLE BIRDIE"
1,12346.0,1.0,22980,PANTRY SCRUBBING BRUSH
2,12346.0,1.0,22982,PANTRY PASTRY BRUSH
3,12347.0,0.0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS
4,12347.0,0.0,85099B,JUMBO BAG RED RETROSPOT


In [9]:
recommendation_df.to_csv('/content/customer_recommendations.csv', index=False)
